# A single-row domain
For some applications of the EGM Framework, a single-row domain is the most efficient setup. The code in this notebook will create a 1000 m long domain using 10x10 cells. I recommend to save the files generated here within the Tutorial's directory in folder called **domain02**

In [ ]:
lib_folder = 'C:/Wetlands/Tutorials'        # replace lib_folder with the path to the directory where you
from sys import path                        #have the egmlib.py file
path.append(lib_folder)
from egmlib import *
from IPython.display import HTML

# 1 Setting up the domain properties and special tiles area

In [ ]:
# Creating a variable from EGMproject class
dom = EGMproject()

# Initialise its properties based on a given number of rows and columns
dom.initialize_from_dimensions(nrows=1, ncols=102)
print('Number of cells', dom.NoC)
print('Number of links', dom.NoL)

# Set dx = dy = 10 m to all cells in the domain. These attributes are arrays with (nrows,ncols) shape
dom.dx[:,:] = 10.
dom.dy[:,:] = 10.

# Select the first two columns as "INPUT_CELLS" area
dom.set_special_tiles("INPUT_CELLS", dom.number[:,0:2])
print('Special areas in the domain:')
display("INPUT_CELLS:", dom.tiles["INPUT_CELLS"]["cells"])

# Re-setting the coordinate system: set x-coordinates to start at -2*dx, as we selected the first two
#columns as INPUT_CELLS area
dom.set_coordinates(xWest=-20)

# Print the cells' number table
numdf = pd.DataFrame(dom.number, columns=dom.x[0,:], index=dom.y[:,0])
print("Coordinates' system:")
display(HTML(numdf.to_html()))

# Update Manning's roughness coefficient to n = 0.30 at all cells
dom.Mann[:,:] = 0.30
dom.Mann[dom.tiles["INPUT_CELLS"]["mask"]] = dom.tiles["INPUT_CELLS"]["roughness"]

# Bottom elevation
dom.z = dom.x * 0.001

# Creating the base map of domain's terrain and cell/links location
f, a = dom.map_domain(fs=(10,2))
pyplot.show()
pyplot.close(f)

# 2 Hillslope Diffusion profiles

In [ ]:
# Finding horizontal and vertical continuos line
dom.build_hillslope_profiles()

xp, yp = dom.x.flatten(), dom.y.flatten()
f, a = pyplot.subplots(figsize=(10,2), dpi=100, constrained_layout=True)
for p in dom.profiles:
    a.plot(xp[p], yp[p], 'k-', lw=1)
a.plot(xp, yp, 'b.')
a.set_ylabel('y [m]')
a.set_xlabel('x [m]')
pyplot.show()
pyplot.close(f)

# 3 Boundary Conditions
1. Set Input Links
2. Input Series
3. Run-time settings and initial condition

In [ ]:
# HxT boundary locations are composed by [first/inlet cell number, second/outlet cell number, status: 0=inactive;
#1=active]. It must be converted to np.ndarray because of the boundary file writing function
dom.bound_at = np.array([[dom.number[row,0], dom.number[row,1], 1] for row in range(dom.nrows)], dtype=int)
display(dom.bound_at)

In [ ]:
#Calling the sinusoidal wave creator function
time_stamps, levels = sinusoid(amplitude=1.0, mean_level=0.0, wave_length=12*3600, time_step=300,
                               final_time=24*3600)
dom.bound_time_step = 300

# Set water depths at boundary-links
dom.set_boundary_depths(levels)
pyplot.plot(time_stamps/3600, dom.bound_h[:,0], label='input depths')

# Just to compare, let's plot the sinusoidal levels and the water depths at the first boundary-link
pyplot.plot(time_stamps/3600, levels, label='levels')
pyplot.xlabel('hour')
pyplot.ylabel('metres')
pyplot.legend()
pyplot.grid()
pyplot.show()

## EGM water-level series
Differently from `Tutorial 1`, where we created water level series for a set of years by increasing the mean sea level, here I will create series with different tide amplitudes. This will be useful in `Tutorial 3` to investigate the parameters of the Sediment Transport Model (STM).

In [ ]:
# Similar sinusoidal wave with higher amplitude
_, levels2 = sinusoid(amplitude=1.5, final_time=86400)
_, levels3 = sinusoid(amplitude=2.0, final_time=86400)

# Matrix of water levels 
wl = np.vstack((levels, levels2, levels3))
wl = np.transpose(wl)
inp_levels = pd.DataFrame(wl, columns=['1 m', '1.5 m', '2 m'], index=time_stamps)
# display(inp_levels)

# Create a copy of EGM water level input series to use the same column and index names
ssc = inp_levels.copy()
ssc.iloc[:,:] = 37    # Replace all data by a constant value of 37 g/m3

# Plot EGM inputs
inp_levels.plot()
ssc.plot()

In [ ]:
# time-step used in the solution of hydrodynamic model
dom.solution_time_step = [1, 1, 1, 1]

# final time of the hydrodynamic/sediment transport simulation. Computed based on boundary depths series length
dom.hydrodynamic_final_time = [60, 120, 180, dom.bound_time_step * (dom.bound_h.shape[0] - 1)]

# time-step used to record outputs of hydrodynamic model (also used to solve the sediment transport model)
dom.output_time_step = 300

# creating attribute initial_depth
dom.set_initial_depths(0.0)

# 4 Dump information into files

In [ ]:
save_to_folder = 'C:/Wetlands/Tutorials/domain02'
dom.finalize(save_to_folder)

inp_levels.to_csv('C:/Wetlands/Tutorials/water_levels_2.csv')
ssc.to_csv('C:/Wetlands/Tutorials/sediment_conc_2.csv')